In [1]:
'''from google.colab import drive
drive.mount("/content/gdrive")'''

'from google.colab import drive\ndrive.mount("/content/gdrive")'

# **Prepare**

In [2]:
!pip install transformers==3.0.2
'''!pip install transformers==2.9
'''
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install nltk
!pip install Keras-Preprocessing

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/transformers-3.0.2+computecanada-py3-none-any.whl
  Using cached tokenizers-0.8.1rc1.tar.gz (97 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/sacremoses-0.0.49+computecanada-py3-none-any.whl
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      /tmp/pip-build-env-awoswh_r/overlay/lib/python3.10/site-packages/setuptools/dist.py:534: UserWarning: Normalizing '0.8.1.rc1' to '0.8.1rc1'
        warnings.warn(tmpl.format(**locals())

In [3]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import math
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer

import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import BertTokenizer
from transformers import DistilBertTokenizer, RobertaTokenizer
from tensorflow.keras.optimizers import Adam, SGD
from transformers import TFXLNetModel, XLNetTokenizer
import nltk
import re
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from numpy import array
from keras.preprocessing.text import one_hot
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from tensorflow.keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers import Concatenate
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from tqdm import tqdm, trange


/project/6001568/zhe8nov1/jp1/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-06 20:00:04.268480: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 20:00:08.538755: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-06 20:00:33.894437: W tensorflow/compiler/xla/stream_executor/platform/d

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
train = pd.read_csv("train.scruples-anecdotes.csv" ,skip_blank_lines=True)

dev =pd.read_csv("dev-scruples-anecdotes.csv", skip_blank_lines=True)

test =pd.read_csv("test.scruples-anecdotes.csv", skip_blank_lines=True)


In [6]:
print(train['text'].isnull().sum())
print(dev['text'].isnull().sum())
print(test['text'].isnull().sum())

22
1
4


In [7]:
train = train.dropna()
dev = dev.dropna()
test = test.dropna()

In [8]:
print(train['text'].isnull().sum())
print(dev['text'].isnull().sum())
print(test['text'].isnull().sum())

0
0
0


In [9]:
#train_50Percent = train.sample(frac=0.50, random_state=42)
train_df = train[['text','binarized_label']]

In [10]:
dev_df = dev[['text','binarized_label']]

In [11]:
test_df =  test[['text','binarized_label']]

In [12]:
dev_df

,text,binarized_label
0,My boyfriend (well just call him M) and I are ...,RIGHT
2,\n\nMy really good friend is getting married....,RIGHT
3,"I've been very lonely for a long time, few fri...",RIGHT
4,Alt account because friends know my real one. ...,RIGHT
5,Pretty much was hanging out with two friends. ...,WRONG
...,...,...
2494,For reference we're both juniors at a T20 coll...,WRONG
2495,So this just happened and I'm a little taken a...,WRONG
2497,"So, I've been friends with someone for a few y...",RIGHT
2498,My first post!\n\nNo words were exchanged in t...,RIGHT


In [13]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

label_encoder = preprocessing.LabelEncoder()

train_df['num_binarized_label']= label_encoder.fit_transform(train_df['binarized_label'])
dev_df['num_binarized_label']= label_encoder.fit_transform(dev_df['binarized_label'])
test_df['num_binarized_label']= label_encoder.fit_transform(test_df['binarized_label'])


In [14]:
train_df

,text,binarized_label,num_binarized_label
0,"Backstory: So, I got an Xbox one for Christmas...",RIGHT,0
1,I work with about six other people at might jo...,RIGHT,0
2,Context: There was an Instagram post about unp...,RIGHT,0
3,Me and my friends spent sometime organizing a ...,WRONG,1
4,A little background. I'm a far from rich guy w...,RIGHT,0
...,...,...,...
27761,So a bit of background info. My girlfriend has...,RIGHT,0
27762,Context:\n\nI’ve been best friends with this g...,RIGHT,0
27763,So me (19M) and my gf’s (18F) relationship has...,RIGHT,0
27764,"A little info, I’m an Early College student, f...",RIGHT,0


In [15]:
nbl1 = train_df['num_binarized_label'] == 1
train_df_1 = pd.DataFrame(train_df[nbl1])
train_df_1

,text,binarized_label,num_binarized_label
3,Me and my friends spent sometime organizing a ...,WRONG,1
18,A girl I dated a few years ago and I have rema...,WRONG,1
22,TLDR at bottom.\n\nSo went to a nice dinner wi...,WRONG,1
27,"For some context, I live in a flatshare with t...",WRONG,1
30,I (24M) have been with my girlfriend(29F) for ...,WRONG,1
...,...,...,...
27742,It all started when I gave her the password to...,WRONG,1
27743,"Bit of a backstory, my boyfriend and I began d...",WRONG,1
27747,I work in a very small team at my company. So ...,WRONG,1
27751,"On Valentine’s Day, my GF and I had a great di...",WRONG,1


In [16]:
nbl0 = train_df['num_binarized_label'] == 0
train_df_new = pd.DataFrame(train_df[nbl0]) # dataframe that only contains '0' as num_binarized_label
#train_df_new
train_df_0 = train_df_new.iloc[:5738]
train_df_0

,text,binarized_label,num_binarized_label
0,"Backstory: So, I got an Xbox one for Christmas...",RIGHT,0
1,I work with about six other people at might jo...,RIGHT,0
2,Context: There was an Instagram post about unp...,RIGHT,0
4,A little background. I'm a far from rich guy w...,RIGHT,0
5,TL;DR: sister's husband recently purchased a (...,RIGHT,0
...,...,...,...
7799,"*Mobile, sorry in advance\n\nBackground: I’m a...",RIGHT,0
7801,So this is pretty long so I’ll get right into ...,RIGHT,0
7802,Right then: first the setting. My condo has al...,RIGHT,0
7803,Got into an argument with my dad about eating ...,RIGHT,0


In [17]:
balance_train_df = train_df_0.append(train_df_1, ignore_index=True)
#balance_train_df
train_df = balance_train_df.sample(frac = 1) # shuffle

In [18]:
train_df

,text,binarized_label,num_binarized_label
11016,Greetings. will probably delete this post beca...,WRONG,1
1862,So I had this friend that was heavy into drugs...,RIGHT,0
6131,"So my mom never developed good eating habits, ...",WRONG,1
3020,My dad and his wife have a vacation condo at t...,RIGHT,0
3637,"So, I recently had surgery on my foot, I had a...",RIGHT,0
...,...,...,...
3139,I hope this I see the right type of post for t...,RIGHT,0
3324,"Hey AITA, long time first time, etc etc. I am ...",RIGHT,0
926,"So, some background, i am 14m so am obviously ...",RIGHT,0
7881,"Now, this already sounds pretty bad for me, bu...",WRONG,1


In [19]:
train_df

,text,binarized_label,num_binarized_label
11016,Greetings. will probably delete this post beca...,WRONG,1
1862,So I had this friend that was heavy into drugs...,RIGHT,0
6131,"So my mom never developed good eating habits, ...",WRONG,1
3020,My dad and his wife have a vacation condo at t...,RIGHT,0
3637,"So, I recently had surgery on my foot, I had a...",RIGHT,0
...,...,...,...
3139,I hope this I see the right type of post for t...,RIGHT,0
3324,"Hey AITA, long time first time, etc etc. I am ...",RIGHT,0
926,"So, some background, i am 14m so am obviously ...",RIGHT,0
7881,"Now, this already sounds pretty bad for me, bu...",WRONG,1


In [20]:
def clean_text(text):
    #Remove emojis and special chars
    clean=text
    #reg = re.compile('\\.+?(?=\B|$)')
    #clean = text.apply(lambda r: re.sub(reg, string=r, repl=''))
    #reg = re.compile('\x89Û_')
    #clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    reg = re.compile('\&amp')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='&'))
    reg = re.compile('\\n')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    
    #Remove hashtag symbol (#)
    #clean = clean.apply(lambda r: r.replace('#', ''))

    #Remove user names
    reg = re.compile('@[a-zA-Z0-9\_]+')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='@'))

    #Remove URLs
    reg = re.compile('https?\S+(?=\s|$)')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='www'))

    #Lowercase
    #clean = clean.apply(lambda r: r.lower())
    return clean

In [21]:
train_df = train_df.replace(np.nan, '', regex=True)

dev_df = dev_df.replace(np.nan, '', regex=True)

test_df = test_df.replace(np.nan, '', regex=True)


In [22]:
train_df['clean'] = clean_text(train_df['text'])
dev_df['clean'] = clean_text(dev_df['text'])
test_df['clean'] = clean_text(test_df['text'])

In [23]:
train_df

,text,binarized_label,num_binarized_label,clean
11016,Greetings. will probably delete this post beca...,WRONG,1,Greetings. will probably delete this post beca...
1862,So I had this friend that was heavy into drugs...,RIGHT,0,So I had this friend that was heavy into drugs...
6131,"So my mom never developed good eating habits, ...",WRONG,1,"So my mom never developed good eating habits, ..."
3020,My dad and his wife have a vacation condo at t...,RIGHT,0,My dad and his wife have a vacation condo at t...
3637,"So, I recently had surgery on my foot, I had a...",RIGHT,0,"So, I recently had surgery on my foot, I had a..."
...,...,...,...,...
3139,I hope this I see the right type of post for t...,RIGHT,0,I hope this I see the right type of post for t...
3324,"Hey AITA, long time first time, etc etc. I am ...",RIGHT,0,"Hey AITA, long time first time, etc etc. I am ..."
926,"So, some background, i am 14m so am obviously ...",RIGHT,0,"So, some background, i am 14m so am obviously ..."
7881,"Now, this already sounds pretty bad for me, bu...",WRONG,1,"Now, this already sounds pretty bad for me, bu..."


In [24]:
test_df['clean'][0]

"   Throwaway for obvious reasons.     I dropped out of uni 2 years ago because I was planning on moving to a different country and going to Uni there. However, my parents didn't want to help me in any way and I couldn't afford it myself. They've always supported my younger sister though, bought her an apartment and they're even paying her bills, even though she earns enough money to do that herself. So atm I'm stuck at home with them, which sucks as I am 25 yo and I feel like I'm wasting my life completely.     Both my parents were extremely abusive, especially my dad. As a result, I have social anxiety, trust issues, panic attacks etc. I can deal with everything as I got used to it anyway, but lately things have gotten worse.     A few years ago my dad was diagnosed with type II diabetes, and he retired. He's been staying at home ever since and he's slowly but surely becoming a vegetable. He had a stroke which left him unable to speak/walk properly. I'm no doctor, but I found his hos

# **Data preprocessing**

In [25]:
train_df["lower"] = train_df["clean"].apply(lambda x: x.lower())

dev_df["lower"] = dev_df["clean"].apply(lambda x: x.lower())

test_df["lower"] = test_df["clean"].apply(lambda x: x.lower())
display(train_df.head())

,text,binarized_label,num_binarized_label,clean,lower
11016,Greetings. will probably delete this post beca...,WRONG,1,Greetings. will probably delete this post beca...,greetings. will probably delete this post beca...
1862,So I had this friend that was heavy into drugs...,RIGHT,0,So I had this friend that was heavy into drugs...,so i had this friend that was heavy into drugs...
6131,"So my mom never developed good eating habits, ...",WRONG,1,"So my mom never developed good eating habits, ...","so my mom never developed good eating habits, ..."
3020,My dad and his wife have a vacation condo at t...,RIGHT,0,My dad and his wife have a vacation condo at t...,my dad and his wife have a vacation condo at t...
3637,"So, I recently had surgery on my foot, I had a...",RIGHT,0,"So, I recently had surgery on my foot, I had a...","so, i recently had surgery on my foot, i had a..."


In [26]:
!pip install contractions
import contractions

train_df["lower"] = train_df["lower"].apply(lambda x: contractions.fix(x))
dev_df["lower"] = dev_df["lower"].apply(lambda x: contractions.fix(x))
test_df["lower"] = test_df["lower"].apply(lambda x: contractions.fix(x))

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


In [27]:
def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r'', text) # or ''.join([x for x in text if x in string.printable]) 

# remove non-ascii characters from the text

train_df["lower"] = train_df["lower"].apply(lambda x: remove_non_ascii(x))
dev_df["lower"] = dev_df["lower"].apply(lambda x: remove_non_ascii(x))
test_df["lower"] = test_df["lower"].apply(lambda x: remove_non_ascii(x))

In [28]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# remove non-ascii characters from the text

train_df["lower"] = train_df["lower"].apply(lambda x: remove_special_characters(x))
dev_df["lower"] = dev_df["lower"].apply(lambda x: remove_special_characters(x))
test_df["lower"] = test_df["lower"].apply(lambda x: remove_special_characters(x))

In [29]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/zhe8nov1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
# Tokenizing the tweet base texts.
from nltk.tokenize import word_tokenize

train_df['tokenized'] = train_df['lower'].apply(word_tokenize)
dev_df['tokenized'] = dev_df['lower'].apply(word_tokenize)
test_df['tokenized'] = test_df['lower'].apply(word_tokenize)

In [31]:
from nltk.stem import SnowballStemmer

def snowball_stemmer(text):
    """
        Stem words in list of tokenized words with SnowballStemmer
    """
    stemmer = nltk.SnowballStemmer("english")
    stems = [stemmer.stem(i) for i in text]
    return stems


In [32]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/zhe8nov1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/zhe8nov1/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [33]:
from nltk.stem import WordNetLemmatizer

def lemmatize_word(text):
    """
        Lemmatize the tokenized words
    """

    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma


lemmatizer = WordNetLemmatizer()

train_df['lemmatize_word_wo_pos'] = train_df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


dev_df['lemmatize_word_wo_pos'] = dev_df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


test_df['lemmatize_word_wo_pos'] = test_df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


In [34]:
connect1 = []
for i in train_df['lemmatize_word_wo_pos']:
  connect1.append(' '.join(i))
train_df['clean1'] = connect1


connect2 = []
for i in dev_df['lemmatize_word_wo_pos']:
  connect2.append(' '.join(i))
dev_df['clean1'] = connect2


connect3 = []
for i in test_df['lemmatize_word_wo_pos']:
  connect3.append(' '.join(i))
test_df['clean1'] = connect3



In [35]:
dev_df['clean'][0]

'My boyfriend (well just call him M) and I are creators on Instagram (he is also on TikTok). I\'m a digital and traditional artist and in one of my classes we create t shirts and things (socks, lanyards, water bottles, stickers, you name it). I\'m a big fan of putting my art on t shirts so I can sell them.  M wanted to make some posts for TikTok and wanted to use the shirt I made today. My one personal rule is that if the shirt isn\'t being given to someone else, then I wear it first (this is because I know I wont get it back. Ive made one other shirt that I\'ve worn, and he wore it for a week and hasn\'t given it back.) So I told him no, because I haven\'t worn the shirt yet.   M proceeded to complain about not having content to post, and how his IG post today didn\'t get any engagement, saying "of course you dont know, because you never look."  Am I the asshole for wanting to wear the shirt I made just once?'

In [36]:
dev_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1
0,My boyfriend (well just call him M) and I are ...,RIGHT,0,My boyfriend (well just call him M) and I are ...,my boyfriend (well just call him m) and i are ...,"[my, boyfriend, (, well, just, call, him, m, )...","[my, boyfriend, (, well, just, call, him, m, )...",my boyfriend ( well just call him m ) and i ar...
2,\n\nMy really good friend is getting married....,RIGHT,0,My really good friend is getting married. Y...,my really good friend is getting married. y...,"[my, really, good, friend, is, getting, marrie...","[my, really, good, friend, is, getting, marrie...",my really good friend is getting married . yay...
3,"I've been very lonely for a long time, few fri...",RIGHT,0,"I've been very lonely for a long time, few fri...","i have been very lonely for a long time, few f...","[i, have, been, very, lonely, for, a, long, ti...","[i, have, been, very, lonely, for, a, long, ti...","i have been very lonely for a long time , few ..."
4,Alt account because friends know my real one. ...,RIGHT,0,Alt account because friends know my real one. ...,alt account because friends know my real one. ...,"[alt, account, because, friends, know, my, rea...","[alt, account, because, friend, know, my, real...",alt account because friend know my real one . ...
5,Pretty much was hanging out with two friends. ...,WRONG,1,Pretty much was hanging out with two friends. ...,pretty much was hanging out with two friends. ...,"[pretty, much, was, hanging, out, with, two, f...","[pretty, much, wa, hanging, out, with, two, fr...",pretty much wa hanging out with two friend . i...
...,...,...,...,...,...,...,...,...
2494,For reference we're both juniors at a T20 coll...,WRONG,1,For reference we're both juniors at a T20 coll...,for reference we are both juniors at a t20 col...,"[for, reference, we, are, both, juniors, at, a...","[for, reference, we, are, both, junior, at, a,...",for reference we are both junior at a t20 coll...
2495,So this just happened and I'm a little taken a...,WRONG,1,So this just happened and I'm a little taken a...,so this just happened and i am a little taken ...,"[so, this, just, happened, and, i, am, a, litt...","[so, this, just, happened, and, i, am, a, litt...",so this just happened and i am a little taken ...
2497,"So, I've been friends with someone for a few y...",RIGHT,0,"So, I've been friends with someone for a few y...","so, i have been friends with someone for a few...","[so, ,, i, have, been, friends, with, someone,...","[so, ,, i, have, been, friend, with, someone, ...","so , i have been friend with someone for a few..."
2498,My first post!\n\nNo words were exchanged in t...,RIGHT,0,My first post! No words were exchanged in thi...,my first post! no words were exchanged in thi...,"[my, first, post, !, no, words, were, exchange...","[my, first, post, !, no, word, were, exchanged...",my first post ! no word were exchanged in this...


In [37]:
train_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1
11016,Greetings. will probably delete this post beca...,WRONG,1,Greetings. will probably delete this post beca...,greetings. will probably delete this post beca...,"[greetings, ., will, probably, delete, this, p...","[greeting, ., will, probably, delete, this, po...",greeting . will probably delete this post beca...
1862,So I had this friend that was heavy into drugs...,RIGHT,0,So I had this friend that was heavy into drugs...,so i had this friend that was heavy into drugs...,"[so, i, had, this, friend, that, was, heavy, i...","[so, i, had, this, friend, that, wa, heavy, in...",so i had this friend that wa heavy into drug ....
6131,"So my mom never developed good eating habits, ...",WRONG,1,"So my mom never developed good eating habits, ...","so my mom never developed good eating habits, ...","[so, my, mom, never, developed, good, eating, ...","[so, my, mom, never, developed, good, eating, ...","so my mom never developed good eating habit , ..."
3020,My dad and his wife have a vacation condo at t...,RIGHT,0,My dad and his wife have a vacation condo at t...,my dad and his wife have a vacation condo at t...,"[my, dad, and, his, wife, have, a, vacation, c...","[my, dad, and, his, wife, have, a, vacation, c...",my dad and his wife have a vacation condo at t...
3637,"So, I recently had surgery on my foot, I had a...",RIGHT,0,"So, I recently had surgery on my foot, I had a...","so, i recently had surgery on my foot, i had a...","[so, ,, i, recently, had, surgery, on, my, foo...","[so, ,, i, recently, had, surgery, on, my, foo...","so , i recently had surgery on my foot , i had..."
...,...,...,...,...,...,...,...,...
3139,I hope this I see the right type of post for t...,RIGHT,0,I hope this I see the right type of post for t...,i hope this i see the right type of post for t...,"[i, hope, this, i, see, the, right, type, of, ...","[i, hope, this, i, see, the, right, type, of, ...",i hope this i see the right type of post for t...
3324,"Hey AITA, long time first time, etc etc. I am ...",RIGHT,0,"Hey AITA, long time first time, etc etc. I am ...","hey aita, long time first time, etc etc. i am ...","[hey, aita, ,, long, time, first, time, ,, etc...","[hey, aita, ,, long, time, first, time, ,, etc...","hey aita , long time first time , etc etc . i ..."
926,"So, some background, i am 14m so am obviously ...",RIGHT,0,"So, some background, i am 14m so am obviously ...","so, some background, i am 14m so am obviously ...","[so, ,, some, background, ,, i, am, 14m, so, a...","[so, ,, some, background, ,, i, am, 14m, so, a...","so , some background , i am 14m so am obviousl..."
7881,"Now, this already sounds pretty bad for me, bu...",WRONG,1,"Now, this already sounds pretty bad for me, bu...","now, this already sounds pretty bad for me, bu...","[now, ,, this, already, sounds, pretty, bad, f...","[now, ,, this, already, sound, pretty, bad, fo...","now , this already sound pretty bad for me , b..."


# **Random forest**

In [38]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3), lowercase=True)
'''text_transformer = CountVectorizer(input='content',
            encoding='utf-8',
            decode_error='strict',
            preprocessor=None,
            tokenizer=None,
            token_pattern=r'(?u)\b\w\w+\b',
            max_features=None,
            vocabulary=None,
            binary=True)
'''

"text_transformer = CountVectorizer(input='content',\n            encoding='utf-8',\n            decode_error='strict',\n            preprocessor=None,\n            tokenizer=None,\n            token_pattern=r'(?u)\x08\\w\\w+\x08',\n            max_features=None,\n            vocabulary=None,\n            binary=True)\n"

In [39]:
train_x = train_df[["clean1"]]
train_y = train_df[["num_binarized_label"]]

dev_x = dev_df[["clean1"]]
dev_y = dev_df[["num_binarized_label"]]

test_x = test_df[["clean1"]]
test_y = test_df[["num_binarized_label"]]

In [40]:
train_x_text = text_transformer.fit_transform(train_x['clean1'])
dev_x_text = text_transformer.transform(dev_x['clean1'])
test_x_text = text_transformer.transform(test_x['clean1'])

In [41]:
train_x_text.shape

(11476, 2142130)

In [42]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

classifier=RandomForestClassifier()
classifier.fit(train_x_text,train_y)


RandomForestClassifier()

In [44]:
y_pred2 = classifier.predict(test_x_text)




from sklearn.metrics import classification_report
# Making the Confusion Matrix
print(confusion_matrix(test_y,y_pred2)) 
print(classification_report(test_y, y_pred2)) 

[[1170  657]
 [ 246  285]]
              precision    recall  f1-score   support

           0       0.83      0.64      0.72      1827
           1       0.30      0.54      0.39       531

    accuracy                           0.62      2358
   macro avg       0.56      0.59      0.55      2358
weighted avg       0.71      0.62      0.65      2358



In [45]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 20:10:01
